In [1]:
import pandas
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
import numpy as np
from praveganb.idracdemo import *
import seaborn as sns
%matplotlib inline

In [2]:
def display_data(metric_report, metric_id=None, mean=False):
    data = []
    for metric in metric_report['MetricValues']:
        try:
            metric['MetricValue'] = float(metric['MetricValue'])
        except:
            pass
        data.append(metric)
    if metric_id:
        data = [metric for metric in data if metric.get('MetricId') == metric_id]
    df = pandas.json_normalize(data)
    if mean:
         display(df.groupby('MetricId')['MetricValue'].mean())
    else:
        display(df)

In [3]:
idracdata = IdracData(scope='idracdemo', stream='idracdata', gateway='pravega-grpc-gateway:80')

In [4]:
pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)
pandas.set_option('display.width', None)
pandas.set_option('display.max_colwidth', None)
stream_info = idracdata.get_stream_start_end()
print('Start: {0}'.format(stream_info[0]))
print('End: {0}'.format(stream_info[1]))

Start: H4sIAAAAAAAAADOwykwpSkxOSc3N14ewEksSrQx0DHWMgCQQQkgAg7OsXCcAAAA=
End: H4sIAAAAAAAAACWIQQqAMAwEPxRw025syG+C7cGDCOL/MSADwzCIcz55zHXd21/5ZkBUWrkIVbg7yWG1yd6HAw4VtR3GsjV8+pIZXEgAAAA=


In [ ]:
import collections
data_ids = [
    'NICStatistics',
    'CUPS',
    'ThermalSensor',
    'CPUMemMetrics',
    'CPUSensor',
    'FanSensor',
    'GPUMetrics',
    'MemorySensor',
    'NicSensor',
    'NICStatistics',
    'NVMeSMARTData',
    'PSUMetrics',
    'PowerMetrics',
    'Sensor',
    'StorageDiskSMARTData',
    'StorageSensor',
    'ThermalMetrics',
    'ThermalSensor'
]
metric_id = None
rack = 'eaglemonk'

# metric_report = idracdata.get_metric_report_from_idrac(from_stream_cut="H4sIAAAAAAAAACXGSQqAQAwF0QsFzGASf24TbBcuRBDvjw1S8Ciuczy9j+O6l//67WIS0ums3Ng2VVEDJUeEIFcYZSgDDvX8AKtFartFAAAA",
#                                                        to_stream_cut="H4sIAAAAAAAAACXGSwqAMAxF0Q0FbH59aXYTrAMHIoj7x4JcONyW53xqn8d1b//VW9mISZardDURC3EYgRmKEc5K6PCQFqPjA26ZTCRFAAAA",
#                                                        # data_id=data_id,
#                                                        rack_label=rack,
#                                                       )
metric_report = idracdata.get_metric_report_from_idrac(from_stream_cut=stream_info[0],
                                                       to_stream_cut=stream_info[1],
                                                       rack_label=rack,
                                                      )
    
grouped = collections.defaultdict(list)
for event in metric_report:
    grouped[event['Id']].append(event)

for _id, events in grouped.items():
    dfs = pandas.DataFrame()
    for event in events:
        clear_output(wait=True)
        df = pandas.json_normalize(event['MetricValues'])
        df['Id'] = event['Id']
        df['Name'] = event['Name']
        df['Timestamp'] = event['Timestamp']
        df['Timestamp'] = pandas.to_datetime(df['Timestamp'],
                           format="%Y-%m-%dT%H:%M:%S.%fZ", utc=True)
        df['Timestamp'] = df['Timestamp'].dt.tz_convert(None)
        # df = df.groupby(['Timestamp', 'Id', 'Name', 'MetricId', 'Oem.Dell.ContextID']).aggregate({'MetricValue': 'max'})
        df = df.groupby(['Timestamp', 'Id', 'MetricId']).aggregate({'MetricValue': 'max'})

        display(df)
        dfs = dfs.append(df)
    dfs.to_csv("IDRACData{0}.csv".format(_id))

In [16]:
! pip install zipfile36

In [23]:
from zipfile import ZipFile
import os
zipObj = ZipFile('idracdata.zip', 'w')

files = [f for f in os.listdir('.') if os.path.isfile(f)]
for filename in files:
    if '.csv' in filename:
        zipObj.write(filename)
        print(filename)
zipObj.close()


IDRACDataNicSensor.csv
IDRACDataStorageSensor.csv
IDRACDataPSUMetrics.csv
IDRACDataNICStatistics.csv
IDRACDataStorageDiskSMARTData.csv
IDRACDataSensor.csv
IDRACDataNVMeSMARTData.csv
IDRACDataThermalSensor.csv
IDRACDataCPUMemMetrics.csv
IDRACDataFanSensor.csv
IDRACDataPowerMetrics.csv
IDRACDataMemorySensor.csv
IDRACDataCPUSensor.csv
IDRACDataGPUMetrics.csv
IDRACDataThermalMetrics.csv
IDRACDataCUPS.csv


In [7]:
display(dfs)

""


In [11]:
df.to_csv("IDRACData{0}.csv".format(data_id))

In [ ]:
# display_data(metric_report)
# display_data(metric_report, metric_id='WattsReading')
# RPMReading
calculate_mean = True
display_data(metric_report, metric_id=metric_id, mean=calculate_mean)

In [ ]:
idracalert = IdracData(scope='idracdemo', stream='alerts', gateway='pravega-grpc-gateway:80')

In [ ]:
stream_info = idracalert.get_stream_start_end()
stream_info

In [ ]:
data = []
_events = idracalert.get_events(stream_info[0])
for i in _events:
    clear_output(wait=True)
    for event in i['Events']:
        event['remoteaddr'] = i['RemoteAddr']
        data.append(event)
    df = pandas.json_normalize(data)
    display(df)